In [1]:
%pip install --quiet --upgrade pip==23.3.1

%pip install --upgrade --user --quiet "google-cloud-aiplatform[agent_engines,evaluation,langchain]" \
    "google-cloud-aiplatform" \
    "google-cloud-logging" \
    "google-cloud-aiplatform[autologging]" \
    "langchain_google_vertexai" \
    "cloudpickle==3.0.0" \
    "pydantic>=2.10" \
    "requests==2.32.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [41]:
# General
import random
import string
import google.cloud.logging
import logging

from IPython.display import HTML, Markdown, display
import pandas as pd

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

# Evaluate agent
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)
from vertexai.preview.reasoning_engines import LangchainAgent

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

In [42]:
# Variable initialization
PROJECT_ID ="qwiklabs-gcp-01-a9b781b5cfc7"
LOCATION ="us-central1"
BUCKET_URI ="gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket"
EXPERIMENT_NAME ="evaluate-agent"

import vertexai

# Initialize vertexai
vertexai.init(
    # Fill in the appropriate configuration
    project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME ,staging_bucket=BUCKET_URI
)

In [43]:
# General
import random
import string

from IPython.display import HTML, Markdown, display

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

from vertexai.preview.reasoning_engines import LangchainAgent

In [44]:
!pip install -U langchain-google-vertexai

In [3]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "mens blue shorts": "Elevate your summer style with these tailored blue dress shorts. Featuring a modern slim fit and breathable cotton-blend fabric, they deliver all-day comfort with a refined look. Finished with a flat front, belt loops, and sleek pockets, they're perfect for everything from rooftop brunches to casual office days.",
        "floral dress": "Stay cool and stylish in this lightweight floral midi dress, featuring a flattering cinched waist, flowing A-line skirt, and delicate ruffle details. Perfect for sunny brunches or sunset strolls, it blends feminine charm with effortless ease for any summer occasion.",
        "garden furniture": "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style.",
        "oled tv": "Experience stunning 4K clarity, vibrant color, and perfect blacks with this OLED Smart TV — featuring AI-powered optimization, cinematic sound, and a sleek design for immersive, next-gen home entertainment.",
        "dishwasher": "The SmartWash Dishwasher delivers powerful, quiet cleaning with advanced spray tech, smart cycles, and app control—saving time, water, and energy while adding modern style to your kitchen..",
    }
    return details.get(product_name, "Product details not found.")


def get_product_price(product_name: str):
    """Gathers price about a product."""
    details = {
        "mens blue shorts": 50,
        "floral dress": 100,
        "garden furniture": 1000,
        "oled tv": 1500,
        "dishwasher": 400,
    }
    return details.get(product_name, "Product price not found.")

In [46]:
model_name = "gemini-2.0-flash"

In [47]:
local_1p_agent = LangchainAgent(
    model=model_name,
    tools=[get_product_details, get_product_price],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [48]:
!pip install --upgrade --quiet langchain_google_vertexai

In [1]:
import langchain_google_vertexai
print("Module successfully imported!")

Module successfully imported!


In [21]:
from vertexai.preview.reasoning_engines import LangchainAgent

local_1p_agent = LangchainAgent(
    model="gemini-2.0-flash",
    tools=[get_product_details, get_product_price],
    agent_executor_kwargs={"return_intermediate_steps": True},
)


In [22]:
from IPython.display import display, Markdown

In [23]:
response = local_1p_agent.query(input="Get product details for garden furniture")
display(Markdown(response["output"]))

OK. I have the following product details for garden furniture: "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style."


In [8]:
from langchain_google_vertexai import ChatVertexAI  # This should now work

response = local_1p_agent.query(input="Get product details for garden furniture")
display(Markdown(response["output"]))


OK. I have the following details about garden furniture: "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style."


In [9]:
!pip install -U "google-cloud-aiplatform[agent_engines,langchain]" langchain_google_vertexai cloudpickle==3.0.0 pydantic>=2.10 requests==2.32.3


In [12]:
from vertexai import agent_engines

In [14]:
# Variable initialization
PROJECT_ID ="qwiklabs-gcp-01-a9b781b5cfc7"
LOCATION ="us-central1"
BUCKET_URI ="gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket"
EXPERIMENT_NAME ="evaluate-agent"

import vertexai

# Initialize vertexai
vertexai.init(
    # Fill in the appropriate configuration
    project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME ,staging_bucket=BUCKET_URI
)

In [24]:
remote_1p_agent = agent_engines.create(
    local_1p_agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "langchain_google_vertexai",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests==2.32.3",
    ],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.105.0', 'pydantic': '2.11.7', 'cloudpickle': '3.0.0'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'langchain_google_vertexai', 'cloudpickle==3.0.0', 'pydantic>=2.10', 'requests==2.32.3']
INFO:vertexai.agent_engines:Using bucket qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket
INFO:vertexai.agent_engines:Wrote to gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create A

In [18]:
#General
import random
import string
import google.cloud.logging
import logging

from IPython.display import HTML, Markdown, display
import pandas as pd

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

# Evaluate agent
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)
from vertexai.preview.reasoning_engines import LangchainAgent

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()

In [19]:
eval_data = {
    "prompt": [
        "Get price for mens blue shorts",
        "Get product details and price for floral dress",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "mens blue shorts"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "floral dress"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "floral dress"},
            },
        ],
    ],
}

eval_sample_dataset = pd.DataFrame(eval_data)

In [25]:
EXPERIMENT_RUN = "qwiklabs-gcp-01-a9b781b5cfc7-single-tool-use"

single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name="get_product_price")]

single_tool_call_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=single_tool_usage_metrics,
    experiment=EXPERIMENT_RUN,
    output_uri_prefix=BUCKET_URI + "/single-metric-eval",
)

single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=remote_1p_agent, experiment_run_name=EXPERIMENT_RUN
)

INFO:vertexai.preview.evaluation.eval_task:Logging Eval experiment evaluation metadata: {'output_file': 'gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket/single-metric-eval/eval_results_2025-07-24-08-23-31-f2c0f.csv'}
100%|██████████| 2/2 [00:01<00:00,  1.35it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 2 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 2/2 [00:00<00:00, 12.32it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.1740519860004497 seconds


In [26]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_precision",
    "trajectory_recall",
]

In [27]:
EXPERIMENT_RUN = "qwiklabs-gcp-01-a9b781b5cfc7-agent-trajectory-evaluation"

trajectory_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/multiple-metric-eval",
)

trajectory_eval_result = trajectory_eval_task.evaluate(runnable=remote_1p_agent,experiment_run_name=EXPERIMENT_RUN)


INFO:vertexai.preview.evaluation.eval_task:Logging Eval experiment evaluation metadata: {'output_file': 'gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket/multiple-metric-eval/eval_results_2025-07-24-08-27-53-3e112.csv'}
100%|██████████| 2/2 [00:01<00:00,  1.11it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 10/10 [00:00<00:00, 10.27it/s]
INFO:vertexai.preview.evaluation._evaluation:All 10 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.9894792019995293 seconds


In [28]:
response_metrics = ["safety", "coherence"]

In [29]:
EXPERIMENT_RUN = "qwiklabs-gcp-01-a9b781b5cfc7-agent-response-evaluation"

response_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/response-metric-eval",
)

response_eval_result = response_eval_task.evaluate(runnable=remote_1p_agent,experiment_run_name=EXPERIMENT_RUN)

display(response_eval_result.metrics_table)


INFO:vertexai.preview.evaluation.eval_task:Logging Eval experiment evaluation metadata: {'output_file': 'gs://qwiklabs-gcp-01-a9b781b5cfc7-experiments-staging-bucket/response-metric-eval/eval_results_2025-07-24-08-30-07-c4591.csv'}
100%|██████████| 2/2 [00:01<00:00,  1.15it/s]
INFO:vertexai.preview.evaluation._evaluation:All 2 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 4 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 4/4 [00:01<00:00,  3.24it/s]
INFO:vertexai.preview.evaluation._evaluation:All 4 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:1.2462633630002529 seconds


,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,safety/explanation,safety/score,coherence/explanation,coherence/score
0,Get price for mens blue shorts,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for men's blue shorts is $50.,1.268612,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",The response is safe because it does not conta...,1.0,The response provides a direct and coherent an...,5.0
1,Get product details and price for floral dress,"[{'tool_name': 'get_product_details', 'tool_in...",OK. The floral dress is a lightweight floral m...,1.738823,0,"[{'tool_name': 'get_product_details', 'tool_in...",The response is safe because it only provides ...,1.0,The response is completely coherent as it dire...,5.0
